In [1]:
!pip3 install --upgrade --quiet langchain langchain-community langchain-openai chromadb 
!pip3 install --upgrade --quiet pypdf pandas streamlit python-dotenv


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
# Import Langchain modules
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Other modules and packages
import os
import tempfile
import streamlit as st  
import pandas as pd
from dotenv import load_dotenv

d:\AI-Agents\extract_pdf\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

## Define our LLM

In [26]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", 
    google_api_key=GOOGLE_API_KEY
)
llm.invoke("Tell me a joke about cats")

AIMessage(content='Why are cats such bad dancers?  Because they have two left feet… and two left paws!\n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-89f9afdc-67e0-4c1c-9c0b-2ea539eda3c1-0', usage_metadata={'input_tokens': 6, 'output_tokens': 21, 'total_tokens': 27})

## Process PDF document

### Load PDF document

In [27]:
loader = PyPDFLoader("D:\AI-Agents\extract_pdf\data\Oppenheimer-2006-Applied_Cognitive_Psychology.pdf")
pages = loader.load()
pages

Ignoring wrong pointing object 18 0 (offset 0)


[Document(metadata={'source': 'D:\\AI-Agents\\extract_pdf\\data\\Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'page': 0}, page_content='APPLIED COGNITIVE PSYCHOLOGYAppl. Cognit. Psychol.20: 139–156 (2006)Published online 31 October 2005 in Wiley InterScience(www.interscience.wiley.com) DOI: 10.1002/acp.1178Consequences of Erudite Vernacular Utilized Irrespectiveof Necessity: Problems with Using Long Words NeedlesslyDANIEL M. OPPENHEIMER*Princeton University, USASUMMARYMost texts on writing style encourage authors to avoid overly-complex words. However, a majorityof undergraduates admit to deliberately increasing the complexity of their vocabulary so as to givethe impression of intelligence. This paper explores the extent to which this strategy is effective.Experiments 1–3 manipulate complexity of texts and ﬁnd a negative relationship between complex-ity and judged intelligence. This relationship held regardless of the quality of the original essay, andirrespective of the partici

In [28]:
pdf_path = "D:\AI-Agents\extract_pdf\data\Oppenheimer-2006-Applied_Cognitive_Psychology.pdf"  # Update this path
print(f"Checking if PDF exists at: {pdf_path}")
print(f"File exists: {os.path.exists(pdf_path)}")

if os.path.exists(pdf_path):
    # Get file size
    file_size = os.path.getsize(pdf_path)
    print(f"File size: {file_size} bytes")
else:
    print("❌ PDF file not found! Please check the path.")
    # List files in the current directory
    print("Files in current directory:")
    for file in os.listdir("."):
        if file.endswith('.pdf'):
            print(f"  📄 {file}")

Checking if PDF exists at: D:\AI-Agents\extract_pdf\data\Oppenheimer-2006-Applied_Cognitive_Psychology.pdf
File exists: True
File size: 44271 bytes


In [29]:
try:
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()
    
    print(f"✅ Successfully loaded PDF with {len(pages)} pages")
    
    # Inspect the first page
    if pages:
        first_page = pages[0]
        print(f"\n📄 First page metadata: {first_page.metadata}")
        print(f"📄 First page content length: {len(first_page.page_content)}")
        print(f"📄 First 300 characters of first page:")
        print(first_page.page_content[:300])
        print("...")
        
        # Check if content looks reasonable
        if len(first_page.page_content) < 100:
            print("⚠️ Warning: Page content seems very short. PDF might not be loading correctly.")
        
        # Look for expected content (adjust based on your PDF)
        if "title" in first_page.page_content.lower() or "abstract" in first_page.page_content.lower():
            print("✅ PDF content looks good - found expected terms")
        else:
            print("⚠️ Warning: PDF content might not contain expected terms")
    else:
        print("❌ No pages loaded from PDF")
        
except Exception as e:
    print(f"❌ Error loading PDF: {str(e)}")

Ignoring wrong pointing object 18 0 (offset 0)


✅ Successfully loaded PDF with 3 pages

📄 First page metadata: {'source': 'D:\\AI-Agents\\extract_pdf\\data\\Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'page': 0}
📄 First page content length: 3069
📄 First 300 characters of first page:
APPLIED COGNITIVE PSYCHOLOGYAppl. Cognit. Psychol.20: 139–156 (2006)Published online 31 October 2005 in Wiley InterScience(www.interscience.wiley.com) DOI: 10.1002/acp.1178Consequences of Erudite Vernacular Utilized Irrespectiveof Necessity: Problems with Using Long Words NeedlesslyDANIEL M. OPPENHE
...
⚠️ Warning: PDF content might not contain expected terms


In [30]:
if 'pages' in locals() and pages:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=200,
        length_function=len,
        separators=["\n\n", "\n", " "]
    )
    chunks = text_splitter.split_documents(pages)
    
    print(f"✅ Created {len(chunks)} chunks")
    
    # Inspect first few chunks
    for i, chunk in enumerate(chunks[:3]):  # Show first 3 chunks
        print(f"\n📝 Chunk {i+1}:")
        print(f"   Length: {len(chunk.page_content)}")
        print(f"   Metadata: {chunk.metadata}")
        print(f"   Content preview: {chunk.page_content[:200]}...")
        
    # Check if chunks contain useful content
    total_content = sum(len(chunk.page_content) for chunk in chunks)
    print(f"\n📊 Total content across all chunks: {total_content} characters")
else:
    print("❌ Cannot create chunks - pages not loaded successfully")


✅ Created 11 chunks

📝 Chunk 1:
   Length: 1498
   Metadata: {'source': 'D:\\AI-Agents\\extract_pdf\\data\\Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'page': 0}
   Content preview: APPLIED COGNITIVE PSYCHOLOGYAppl. Cognit. Psychol.20: 139–156 (2006)Published online 31 October 2005 in Wiley InterScience(www.interscience.wiley.com) DOI: 10.1002/acp.1178Consequences of Erudite Vern...

📝 Chunk 2:
   Length: 1497
   Metadata: {'source': 'D:\\AI-Agents\\extract_pdf\\data\\Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'page': 0}
   Content preview: an effort not tobe inﬂuenced by the irrelevant source of ﬂuency, they over-compensate and are biased in the oppositedirection. Implications and applications are discussed. Copyright#2005 John Wiley & ...

📝 Chunk 3:
   Length: 471
   Metadata: {'source': 'D:\\AI-Agents\\extract_pdf\\data\\Oppenheimer-2006-Applied_Cognitive_Psychology.pdf', 'page': 0}
   Content preview: language?’ 86.4% of the sample admitted to having done so. Near

### Split document

In [31]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                            chunk_overlap=200,
                                            length_function=len,
                                            separators=["\n\n", "\n", " "])
chunks = text_splitter.split_documents(pages)

### Create embeddings

In [32]:
def get_embedding_function():
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001", 
        google_api_key=GOOGLE_API_KEY
    )
    return embeddings

embedding_function = get_embedding_function()

test_vector = embedding_function.embed_query("cat")
print(f"Embedding dimension: {len(test_vector)}")

Embedding dimension: 768


In [33]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator(evaluator="embedding_distance", 
                            embeddings=embedding_function)

evaluator.evaluate_strings(prediction="Amsterdam", reference="coffeeshop")

{'score': 0.0998730011025093}

In [34]:
evaluator.evaluate_strings(prediction="Paris", reference="coffeeshop")

{'score': 0.0969643258695102}

### Create vector database

In [35]:
import uuid

def create_vectorstore(chunks, embedding_function, vectorstore_path="vectorstore_db"):
    # Create a list of unique ids for each document based on the content
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]
    
    # Ensure that only unique docs with unique ids are kept
    unique_ids = set()
    unique_chunks = []
    
    for chunk, id in zip(chunks, ids):     
        if id not in unique_ids:       
            unique_ids.add(id)
            unique_chunks.append(chunk)        

    print(f"Number of unique chunks: {len(unique_chunks)}")
    
    # Create a new Chroma database from the documents
    vectorstore = Chroma.from_documents(
        documents=unique_chunks, 
        embedding=embedding_function, 
        ids=list(unique_ids), 
        persist_directory=vectorstore_path
    )

    vectorstore.persist()
    print(f"Vectorstore created at {vectorstore_path} with {vectorstore._collection.count()} documents")
    
    return vectorstore

In [36]:
vectorstore = create_vectorstore(
    chunks=chunks, 
    embedding_function=embedding_function, 
    vectorstore_path="vectorstore_db"
)

Number of unique chunks: 10
Vectorstore created at vectorstore_db with 10 documents


## 2. Query for relevant data

In [37]:

vectorstore = Chroma(
    persist_directory="vectorstore_db", 
    embedding_function=embedding_function
)
print(f"Loaded vectorstore with {vectorstore._collection.count()} documents")

Loaded vectorstore with 10 documents


In [38]:
# Create retriever and get relevant chunks
retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={"k": 5})
relevant_chunks = retriever.invoke("What is the title of the paper?")
relevant_chunks

[Document(metadata={'page': 1, 'source': 'D:\\AI-Agents\\extract_pdf\\data\\Oppenheimer-2006-Applied_Cognitive_Psychology.pdf'}, page_content='Copyright#2005 John Wiley & Sons, Ltd. Appl. Cognit. Psychol. 20: 139–156 (2006)'),
 Document(metadata={'page': 2, 'source': 'D:\\AI-Agents\\extract_pdf\\data\\Oppenheimer-2006-Applied_Cognitive_Psychology.pdf'}, page_content='the same linguistic sense were notreplaced. If two entries were of the same length, the replacement was chosen alphabe-tically. When necessary, minor modiﬁcations were made to the essay to maintain thegrammatical structure of a sentence (e.g. replacing ‘an’ with ‘a’ for replacement wordsbeginning with consonants). A ‘moderately complex’ version of each excerpt was createdusing the same algorithm as above, except replacing only every third applicable word.Examples of the stimuli can be found in the appendix.Each participant received only one excerpt. Participants were informed that the excerptcame from a personal statement 

In [39]:
# Prompt template
PROMPT_TEMPLATE = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.

{context}

---

Answer the question based on the above context: {question}
"""

## 3. Generate responses

In [40]:
# Concatenate context text
context_text = "\n\n---\n\n".join([doc.page_content for doc in relevant_chunks])

# Create prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, 
                                question="What is the title of the paper?")
print(prompt)

Human: 
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.

Copyright#2005 John Wiley & Sons, Ltd. Appl. Cognit. Psychol. 20: 139–156 (2006)

---

the same linguistic sense were notreplaced. If two entries were of the same length, the replacement was chosen alphabe-tically. When necessary, minor modiﬁcations were made to the essay to maintain thegrammatical structure of a sentence (e.g. replacing ‘an’ with ‘a’ for replacement wordsbeginning with consonants). A ‘moderately complex’ version of each excerpt was createdusing the same algorithm as above, except replacing only every third applicable word.Examples of the stimuli can be found in the appendix.Each participant received only one excerpt. Participants were informed that the excerptcame from a personal statement for graduate study in the Stanford English department.They were instructed

In [41]:
llm.invoke(prompt)

AIMessage(content='Consequences of Erudite Vernacular Utilized Irrespective of Necessity: Problems with Using Long Words Needlessly\n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-9660af3b-0544-406d-86f1-73f53931103f-0', usage_metadata={'input_tokens': 1138, 'output_tokens': 22, 'total_tokens': 1160})

### Using Langchain Expression Language

In [42]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm
        )
rag_chain.invoke("What's the title of this paper?")

AIMessage(content='Consequences of Erudite Vernacular Utilized Irrespective of Necessity: Problems with Using Long Words Needlessly\n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-5243d908-ffa3-4596-9e13-223fd478261d-0', usage_metadata={'input_tokens': 1128, 'output_tokens': 22, 'total_tokens': 1150})

### Generate structured responses

In [49]:
class AnswerWithSources(BaseModel):
    """An answer to the question, with sources and reasoning."""
    answer: str = Field(description="Answer to question")
    sources: str = Field(description="Full direct text chunk from the context used to answer the question")
    reasoning: str = Field(description="Explain the reasoning of the answer based on the sources")
    
class ExtractedInfo(BaseModel):
    """Extracted information about the research article"""
    paper_title: AnswerWithSources
    paper_summary: AnswerWithSources
    publication_year: AnswerWithSources
    paper_authors: AnswerWithSources

In [48]:
rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm.with_structured_output(ExtractedInfo)
        )

rag_chain.invoke("Give me the title, summary, publication date, authors of the research paper.")

### Transform response into a dataframe

In [24]:
structured_response = rag_chain.invoke("Give me the title, summary, publication date, authors of the research paper.")
df = pd.DataFrame([structured_response.dict()])

# Transforming into a table with two rows: 'answer' and 'source'
answer_row = []
source_row = []
reasoning_row = []

for col in df.columns:
    answer_row.append(df[col][0]['answer'])
    source_row.append(df[col][0]['sources'])
    reasoning_row.append(df[col][0]['reasoning'])

# Create new dataframe with two rows: 'answer' and 'source'
structured_response_df = pd.DataFrame([answer_row, source_row, reasoning_row], columns=df.columns, index=['answer', 'source', 'reasoning'])
structured_response_df

AttributeError: 'NoneType' object has no attribute 'dict'